In [224]:
# %matplotlib nbagg
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import time
import pickle
from datetime import datetime

sys.path.append("..")
# 自分のpath
os.chdir("/home/nakamura/HAI/main_server/hai")

import coloredlogs, logging
logger = logging.getLogger(__name__)
coloredlogs.install(level='CRITICAL', logger=logger)

import pymongo
from pymongo import MongoClient
from PIL import Image
from matplotlib import pylab as plt
from matplotlib.animation import ArtistAnimation
import matplotlib.animation as animation
import numpy as np
import scipy
from bson.objectid import ObjectId
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from notebooks.utils.utils import visualize, display_latest_image, display_image, print_time, strtime_to_epoch, epoch_to_strtime, UpdateDist, display_two_images
from controllers.learner.img2vec import NNFeatures
from controllers.vectorizer.person2vec import Person2Vec
import controllers.learner.datasets as ds
from controllers.dbreader.hue_koki_dbreader import HueDBReader, pair_images, extract_color, ExprHueDBReader
from controllers.tests.test0106 import Test0106, get_hue_label
from controllers.youtubeplayer import get_youtube_label
from controllers.irkit import get_tv_label

mongo = MongoClient('localhost', 20202).hai
from flask import Flask
app = Flask(__name__)
app.config.from_pyfile(filename="application.cfg")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [215]:
start = strtime_to_epoch("2018-01-06 18:05:00")
# end = strtime_to_epoch("2018-12-17 15:05:00")
end = 10e10

In [217]:
get_hue_label(start, end)

({'hue': [[1515229897.2163348, 0],
   [1515229949.3192773, 2],
   [1515230141.180791, 2],
   [1515230166.805435, 1],
   [1515230256.7558606, 2],
   [1515230283.6463628, 0],
   [1515232918.9161975, 2],
   [1515232932.3959007, 1],
   [1515233001.390852, 0],
   [1515233022.250762, 2],
   [1515237964.844595, 2]]},
 ['電球色', 'オフ', '白色'])

In [221]:
get_youtube_label(start, end)

({'youtube': [[1515236593.1309254, 0],
   [1515236727.6453192, 0],
   [1515238016.384955, 0],
   [1515238054.7300446, 1],
   [1515236206.2311873, 0],
   [1515237131.7843866, 0],
   [1515237501.19113, 0],
   [1515236276.4366357, 1]]},
 ['ホワイトノイズ', 'stop'])

In [225]:
get_tv_label(start, end)

({'TV': [[1515233972.3709023, 0],
   [1515238054.731465, 1],
   [1515233887.0158968, 0],
   [1515233944.8362386, 1],
   [1515233980.1109874, 1],
   [1515237749.7918558, 0],
   [1515237913.3010225, 0]]},
 ['on', 'off'])

In [5]:
hue = Test0106('koki')

In [6]:
epoch_to_strtime(1515227783.59965)

'2018-01-06 17:36:23'

In [4]:
img0_cols = mongo.images.find({'cam_id': 'webcam0', 'time': {'$gt': start, '$lt': end}})
img1_cols = mongo.images.find({'cam_id': 'webcam1', 'time': {'$gt': start, '$lt': end}})
imgs_cols = np.array(pair_images(img0_cols, img1_cols))

In [10]:
prints = mongo.print.find({'time': {'$gt': start, '$lt': end}})

In [11]:
for p in prints:
    print(p)

{'text': 'wow', '_id': ObjectId('5a4fccfa1d41c879d8e1e504'), 'time': 1515179258.990909, 'user_name': 'koki'}
{'text': 'スタート', '_id': ObjectId('5a4fce051d41c879d8e1e581'), 'time': 1515179525.060217, 'user_name': 'koki'}
{'text': 'TV', '_id': ObjectId('5a4fce131d41c879d8e1e58c'), 'time': 1515179539.906976, 'user_name': 'koki'}
{'text': 'ソファで寝る', '_id': ObjectId('5a4fce371d41c879d8e1e59f'), 'time': 1515179575.8342004, 'user_name': 'koki'}
{'text': '外出', '_id': ObjectId('5a4fceae1d41c879d8e1e5eb'), 'time': 1515179694.9625509, 'user_name': 'koki'}
{'text': 'アイスを食べる', '_id': ObjectId('5a4fcf041d41c879d8e1e61f'), 'time': 1515179780.8865013, 'user_name': 'koki'}
{'text': 'スマホ', '_id': ObjectId('5a4fcfa01d41c879d8e1e678'), 'time': 1515179936.7902431, 'user_name': 'koki'}
{'text': 'pc', '_id': ObjectId('5a4fd0d81d41c879d8e1e729'), 'time': 1515180248.8999238, 'user_name': 'koki'}
{'text': 'アイス（机で）', '_id': ObjectId('5a4fd15b1d41c879d8e1e770'), 'time': 1515180379.7111242, 'user_name': 'koki'}
{'te

In [12]:
def get_data():
    X = []
    y = []
    for al in activity_log:
        start = strtime_to_epoch(al['start'])
        end = strtime_to_epoch(al['end'])
        img0_cols = mongo.images.find({'cam_id': 'webcam0', 'time': {'$gt': start, '$lt': end}})
        img1_cols = mongo.images.find({'cam_id': 'webcam1', 'time': {'$gt': start, '$lt': end}})
        imgs_cols = pair_images(img0_cols, img1_cols)
        y.extend([al['activity']] * len(imgs_cols))
        X.extend(imgs_cols)

    return X, y

In [13]:
activity_log = [
    {'activity': 'sleep', 'start': '2018-01-06 04:13:04', 'end': '2018-01-06 04:14:52'},
    {'activity': 'absence', 'start': '2018-01-06 04:15:01', 'end': '2018-01-06 04:16:22'},
    {'activity': 'enjoy', 'start': '2018-01-06 04:16:26', 'end': '2018-01-06 04:23:57'},
    {'activity': 'work', 'start': '2018-01-06 04:24:11', 'end': '2018-01-06 04:25:58'},
    # {'activity': 'enjoy', 'start': '2018-01-06 04:24:11', 'end': '2018-01-06 04:25:58'},
    {'activity': 'sleep', 'start': '2018-01-06 04:28:34', 'end': '2018-01-06 04:33:53'},
]

In [15]:
X, y = get_data()

In [29]:
X[0][0]['pose']

{'body': [], 'face': [], 'hand': []}

In [32]:
vectorizer = Person2Vec()

In [34]:
a, b = vectorizer.vectorize(X)

In [36]:
a.shape

(232, 108)

In [69]:
start = strtime_to_epoch("2018-01-06 04:05:00")
# end = strtime_to_epoch("2018-12-17 15:05:00")
end = 10e10

In [78]:
labels = []
classes = ['on', 'off']
irkit_operations =  mongo.operation.find({'controller': 'IRKit', 'time': {'$gt': start, '$lt': end}})
for irkit_operation in irkit_operations:
    for op in irkit_operation['operation']:
        if ('confirmation' not in op) and op['platform'] == 'irkit':
            labels.append([irkit_operation['time'], classes.index(op['data'][1])])

irkit_confirmations =  mongo.confirmation.find({'platform': 'irkit', 'answer': 'True', 'time': {'$gt': start, '$lt': end}})
for irkit_confirmation in irkit_confirmations:
    if irkit_confirmation['confirmation'] == 'テレビをつけますか?':
        labels.append([irkit_confirmation['time'], classes.index('on')])
    elif irkit_confirmation['confirmation'] == 'テレビをけしますか?':
        labels.append([irkit_confirmation['time'], classes.index('off')])

In [79]:
labels

[[1515233972.3709023, 0],
 [1515238054.731465, 1],
 [1515181780.7417686, 0],
 [1515181823.0669172, 1],
 [1515181953.3063886, 0],
 [1515181965.1250966, 1],
 [1515233887.0158968, 0],
 [1515233944.8362386, 1],
 [1515233980.1109874, 1],
 [1515237749.7918558, 0],
 [1515237913.3010225, 0]]

In [94]:
labels = []

In [117]:
youtube_operations =  mongo.operation.find({'controller': 'YoutubePlayer', 'time': {'$gt': start, '$lt': end}})

In [128]:
youtube_operations.next()

{'_id': ObjectId('5a50af161d41c83c78470fd0'),
 'controller': 'YoutubePlayer',
 'operation': [{'confirmation': 'ホワイトノイズを再生しますか?',
   'data': 'ホワイトノイズ',
   'platform': 'play_youtube'}],
 'time': 1515237142.9429364,
 'user': 'koki'}

In [129]:
classes = set()  
for youtube_operation in youtube_operations:
    for op in youtube_operation['operation']:
        if ('confirmation' not in op) and op['platform'] == 'play_youtube':
            classes.add(op['data'])
            labels.append([youtube_operation['time'], op['data']])
        elif ('confirmation' not in op) and op['platform'] == 'stop_youtube':
            classes.add('stop')
            labels.append([youtube_operation['time'], 'stop'])

In [145]:
youtube_confirmations =  mongo.confirmation.find({'platform': 'play_youtube', 'answer': 'True', 'time': {'$gt': start, '$lt': end}})
for youtube_confirmation in youtube_confirmations:
    classes.add(youtube_confirmation['data'])
    labels.append([youtube_confirmation['time'], youtube_confirmation['data']])
    
youtube_confirmations =  mongo.confirmation.find({'platform': 'stop_youtube', 'answer': 'True', 'time': {'$gt': start, '$lt': end}})
for youtube_confirmation in youtube_confirmations:
    classes.add('stop')
    labels.append([youtube_confirmation['time'], 'stop'])

In [146]:
labels

[[1515236593.1309254, 'ホワイトノイズ'],
 [1515236727.6453192, 'ホワイトノイズ'],
 [1515238016.384955, 'ホワイトノイズ'],
 [1515238016.384955, 'ホワイトノイズ'],
 [1515238054.7300446, 'stop'],
 [1515236206.2311873, 'ホワイトノイズ'],
 [1515237131.7843866, 'ホワイトノイズ'],
 [1515237501.19113, 'ホワイトノイズ'],
 [1515236276.4366357, 'stop']]

In [150]:
ddd = list(classes)

In [151]:
ddd

['ホワイトノイズ', 'stop']

In [148]:
labels2 = [[a, classes_list.index(b) for a, b in labels]]

[[1515236593.1309254, 'ホワイトノイズ'],
 [1515236727.6453192, 'ホワイトノイズ'],
 [1515238016.384955, 'ホワイトノイズ'],
 [1515238016.384955, 'ホワイトノイズ'],
 [1515238054.7300446, 'stop'],
 [1515236206.2311873, 'ホワイトノイズ'],
 [1515237131.7843866, 'ホワイトノイズ'],
 [1515237501.19113, 'ホワイトノイズ'],
 [1515236276.4366357, 'stop']]

In [191]:
start = strtime_to_epoch("2018-01-06 18:05:00")
# end = strtime_to_epoch("2018-12-17 15:05:00")
end = 10e10
hue_operations =  mongo.operation.find({'controller': 'Test0106', 'time': {'$gt': start, '$lt': end}})

In [184]:
classes = set()
labels = []
import json
for hue_operation in hue_operations:
    for op in hue_operation['operation']:
        if ('confirmation' not in op) and op['platform'] == 'hue':
            classes.add(op['data']))
            labels.append([hue_operation['time'], json.loads(op['data'])])

TypeError: unhashable type: 'list'

In [209]:
hoge = hue_operations.next()
hoge

{'_id': ObjectId('5a5093d61d41c879b2ec1de8'),
 'controller': 'Test0106',
 'operation': [{'data': '電気をオフにします', 'platform': 'tts'},
  {'data': '[{"state": {"on": false}, "id": "1"}, {"state": {"on": false}, "id": "2"}, {"state": {"on": false}, "id": "3"}]',
   'platform': 'hue'}],
 'time': 1515230166.805435,
 'user': 'koki'}

In [210]:
a = json.loads(hoge['operation'][1]['data'])
a

[{'id': '1', 'state': {'on': False}},
 {'id': '2', 'state': {'on': False}},
 {'id': '3', 'state': {'on': False}}]

In [203]:
a == json.loads(hoge['operation'][1]['data'])

True

In [204]:
if json.loads(hoge['operation'][1]['data']) == [{'id': '1', 'state': {'bri': 254, 'hue': 14957, 'on': True, 'sat': 141}},
 {'id': '2', 'state': {'bri': 254, 'hue': 14957, 'on': True, 'sat': 141}},
 {'id': '3', 'state': {'bri': 254, 'hue': 14957, 'on': True, 'sat': 141}}]:
    

[{'id': '1', 'state': {'bri': 254, 'hue': 14957, 'on': True, 'sat': 141}},
 {'id': '2', 'state': {'bri': 254, 'hue': 14957, 'on': True, 'sat': 141}},
 {'id': '3', 'state': {'bri': 254, 'hue': 14957, 'on': True, 'sat': 141}}]

In [181]:
classes

{'[{"id": "1", "state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}}, {"id": "2", "state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}}, {"id": "3", "state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}}]',
 '[{"id": "1", "state": {"on": true, "sat": 141, "hue": 14957, "bri": 254}}, {"id": "2", "state": {"on": true, "sat": 141, "hue": 14957, "bri": 254}}, {"id": "3", "state": {"on": true, "sat": 141, "hue": 14957, "bri": 254}}]',
 '[{"id": "1", "state": {"on": true, "sat": 53, "hue": 33016, "bri": 254}}, {"id": "2", "state": {"on": true, "sat": 53, "hue": 33016, "bri": 254}}, {"id": "3", "state": {"on": true, "sat": 53, "hue": 33016, "bri": 254}}]',
 '[{"state": {"hue": 14957, "sat": 141, "bri": 254, "on": true}, "id": "1"}, {"state": {"hue": 14957, "sat": 141, "bri": 254, "on": true}, "id": "2"}, {"state": {"hue": 14957, "sat": 141, "bri": 254, "on": true}, "id": "3"}]',
 '[{"state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}, "id": "1"}, {"state": {"hue": 3301

In [177]:
labels

[[1515229897.2163348,
  '[{"id": "1", "state": {"on": true, "sat": 141, "hue": 14957, "bri": 254}}, {"id": "2", "state": {"on": true, "sat": 141, "hue": 14957, "bri": 254}}, {"id": "3", "state": {"on": true, "sat": 141, "hue": 14957, "bri": 254}}]'],
 [1515229949.3192773,
  '[{"id": "1", "state": {"on": true, "sat": 53, "hue": 33016, "bri": 254}}, {"id": "2", "state": {"on": true, "sat": 53, "hue": 33016, "bri": 254}}, {"id": "3", "state": {"on": true, "sat": 53, "hue": 33016, "bri": 254}}]'],
 [1515230141.180791,
  '[{"state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}, "id": "1"}, {"state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}, "id": "2"}, {"state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}, "id": "3"}]'],
 [1515230166.805435,
  '[{"state": {"on": false}, "id": "1"}, {"state": {"on": false}, "id": "2"}, {"state": {"on": false}, "id": "3"}]'],
 [1515230256.7558606,
  '[{"state": {"hue": 33016, "sat": 53, "bri": 254, "on": true}, "id": "1"}, {"state": {"hue"